In [ ]:
# %load ../../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from datetime import datetime
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
# log.setLevel(logging.INFO)
log.setLevel(logging.DEBUG)

In [ ]:
import zcode.math as zmath
import zcode.plot as zplot

In [ ]:
log.setLevel(log.DEBUG)

In [ ]:
log.level, log.DEBUG, log.INFO

In [ ]:
hard_time = -1.7201236742815582
hard_gamma_inner = -0.03556985639992871
gsmf_phi0 = -3.0564167047947643
gsmf_mchar0 = 12.235955681919318
gsmf_alpha0 = -0.9876847863104197
gpf_zbeta = 0.49637865297766504
gpf_qgamma = -1.3153408817208816
gmt_norm = 5.65479556730525
gmt_zbeta = 0.5038273206522703
mmb_amp = 8.265821569516374
mmb_plaw = 0.25942737980048103
mmb_scatter = 0.20729445938337865

hard_time = (10.0 ** hard_time) * GYR
gmt_norm = gmt_norm * GYR
mmb_amp = (10.0 ** mmb_amp) * MSOL

gsmf = holo.sam.GSMF_Schechter(phi0=gsmf_phi0, mchar0_log10=gsmf_mchar0, alpha0=gsmf_alpha0)
gpf = holo.sam.GPF_Power_Law(qgamma=gpf_qgamma, zbeta=gpf_zbeta)
gmt = holo.sam.GMT_Power_Law(time_norm=gmt_norm, zbeta=gmt_zbeta)
mmbulge = holo.relations.MMBulge_KH2013(mamp=mmb_amp, mplaw=mmb_plaw, scatter_dex=mmb_scatter)

sam = holo.sam.Semi_Analytic_Model(
    gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge,
    shape=20
)
hard = holo.hardening.Fixed_Time.from_sam(
    sam, hard_time, gamma_sc=hard_gamma_inner,
    progress=False
)
pta_dur = 16.03 * YR
nfreqs = 40
hifr = nfreqs/pta_dur
pta_cad = 1.0 / (2 * hifr)
fobs_cents = holo.utils.nyquist_freqs(pta_dur, pta_cad)
fobs_edges = holo.utils.nyquist_freqs_edges(pta_dur, pta_cad)
gwb = sam.gwb(fobs_edges, realize=10, hard=hard)



In [ ]:
SHAPE = None
TIME = 1.0 * GYR

sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)
hard = holo.hardening.Fixed_Time.from_sam(sam, TIME, interpolate_norm=False)

In [ ]:
STEPS = 500

# () start from the hardening model's initial separation
rmax = hard._sepa_init
# (M,) end at the ISCO
rmin = utils.rad_isco(sam.mtot)
# rmin = hard._TIME_TOTAL_RMIN * np.ones_like(sam.mtot)
# Choose steps for each binary, log-spaced between rmin and rmax
extr = np.log10([rmax * np.ones_like(rmin), rmin])
rads = np.linspace(0.0, 1.0, STEPS)[np.newaxis, :]
# (M, X)
rads = extr[0][:, np.newaxis] + (extr[1] - extr[0])[:, np.newaxis] * rads
rads = 10.0 ** rads
# (M, Q, Z, X)
mt, mr, rz, rads = np.broadcast_arrays(
    sam.mtot[:, np.newaxis, np.newaxis, np.newaxis],
    sam.mrat[np.newaxis, :, np.newaxis, np.newaxis],
    sam.redz[np.newaxis, np.newaxis, :, np.newaxis],
    rads[:, np.newaxis, np.newaxis, :]
)
# (X, M*Q*Z)
mt, mr, rz, rads = [mm.reshape(-1, STEPS).T for mm in [mt, mr, rz, rads]]
# (X, M*Q*Z) --- `Fixed_Time.dadt` will only accept this shape
dadt = hard.dadt(mt, mr, rads)
# Integrate (inverse) hardening rates to calculate total lifetime to each separation
times_evo = -utils.trapz_loglog(-1.0 / dadt, rads, axis=0, cumsum=True)


In [ ]:
tt = times_evo[-1, :]/GYR
fig, ax = plot.figax(scale='lin')
print(utils.stats(tt))
kale.dist1d(tt, density=True)
plt.show()